In [ ]:
import os
import pandas as pd
import re
import warnings

from pyvis.network import Network

warnings.filterwarnings('ignore')

In [ ]:
kills = pd.DataFrame(columns=['universe', 'character', 'killed', 'responsible for', 'total kills', 'victims'])
deaths = pd.DataFrame(columns=['universe', 'character', 'responsible party', 'description', 'type'])
edges = pd.DataFrame(columns=['universe', 'killer', 'victim', 'direction', 'description', 'type'])

In [ ]:
for path, subdirs, files in os.walk('Universes'):
    for name in files:
        file = path + '\\' + name
        universe = file.split('\\')[1::2][0]

        tsv = pd.read_csv(file, sep='\t')

        for i in range(len(tsv)):
            character = tsv['character'][i]
            killer = str(tsv['killer'][i]).split(', ')
            responsible_party = str(tsv['responsible party'][i]).split(', ')
            description = tsv['description'][i]
            type = tsv['type'][i]

            if killer == ['nan']:
                killer = []
            if responsible_party == ['nan']:
                responsible_party = []

            killerparty = killer + responsible_party
            if killerparty == ['nan']:
                killerparty = []

            for i in range(len(killerparty)):
                if killerparty[i] in killer:
                    row = [universe, killerparty[i], 1, 0, 1, character]
                    kills.loc[len(kills)] = row
                elif killerparty[i] in responsible_party:
                    row = [universe, killerparty[i], 0, 1, 1, character]
                    kills.loc[len(kills)] = row

                if killerparty[i] in killer:
                    row = [universe, killerparty[i], character, 'direct', description, type]
                    edges.loc[len(edges)] = row
                elif killerparty[i] in responsible_party:
                    row = [universe, killerparty[i], character, 'indirect', description, type]
                    edges.loc[len(edges)] = row

            if killerparty == []:
                row = [universe, '', character, '', description, type]
                edges.loc[len(edges)] = row

            row = [universe, character, killerparty, description, type]
            deaths.loc[len(deaths)] = row

In [ ]:
kills = kills.groupby(by=['universe', 'character']).aggregate({'killed':sum, 'responsible for':sum, 'total kills':sum, 'victims':', '.join})
kills.reset_index(inplace=True)
print('----- the 10 Deadliest Killers  -----')
kills.drop('victims', axis=1).sort_values(by='total kills', ascending=False).head(10)

In [ ]:
print('----- Deadliest Killer per Universe -----')
kills.drop('victims', axis=1).sort_values(by='total kills', ascending=False).drop_duplicates('universe')

In [ ]:
deaths_count = pd.DataFrame(deaths.groupby(by=['universe', 'character']).size()).reset_index()
deaths_count.columns = ['universe', 'character', 'deaths']

kills_count = pd.DataFrame(kills.groupby(by=['universe', 'character']).sum('total kills')).reset_index()
kills_count.drop(['killed', 'responsible for'], axis=1, inplace=True)
kills_count.columns = ['universe', 'character', 'kills']

summary = pd.merge(deaths_count, kills_count, left_on=['universe', 'character'], right_on=['universe', 'character'], how='outer')
summary.fillna(0, inplace=True)
summary.sort_values(by=['universe', 'character'], inplace=True)

summary['ratio'] = summary['kills'] / summary['deaths']
summary.loc[summary['deaths'] == 0, 'ratio'] = 0

summary['score'] = 0

for i in range(len(kills)):
    universe = kills.iloc[i]['universe']
    character = kills.iloc[i]['character']
    count = kills.iloc[i]['total kills']
    victims = kills.iloc[i]['victims'].split(', ')

    summary.loc[(summary['universe'] == universe) & (summary['character'] == character), 'score'] += count
    victims = list(set(victims))

    for victim in victims:
        if victim != '' and victim != character:
            try:
                score = summary.loc[(summary['universe'] == universe) & (summary['character'] == victim), 'kills'].values[0]
            except IndexError:
                score = 0
            summary.loc[(summary['universe'] == universe) & (summary['character'] == character), 'score'] += score

summary['transitivity'] = summary['score'] - summary['kills']

summary = summary.round(decimals=2).astype(object)
summary = summary.astype(str)
summary = summary.replace(to_replace = "\.0+$",value = "", regex = True)

print("----- the 10 Deadliest 'Killer Killers' -----")
summary.sort_values(by='transitivity', ascending=False).head(10)

In [ ]:
print('----- Deadliest Universes -----')
pd.DataFrame(deaths.groupby(by=['universe']).size().sort_values(ascending=False)).reset_index().rename(columns={0:'deaths'})

In [ ]:
deaths.to_csv('deaths.tsv', sep='\t', index=False)
kills.to_csv('kills.tsv', sep='\t', index=False)
summary.to_csv('summary.tsv', sep='\t', index=False)

In [ ]:
edges['killer'] += " [" + edges['universe'] + "]"
edges.loc[edges['killer'].str.startswith(' ['), 'killer'] = ''
edges['victim'] += " [" + edges['universe'] + "]"
edges['color'] = pd.Series()

for i in range(len(edges)):
    type = str(edges.iloc[i]['type']).split(', ')

    if 'variable' in type:
        dependent = True
    else:
        dependent = False

    conditions = ["temporary", "non-canon"]
    if re.search('|'.join(conditions), str(type)):
        alive = True
    else:
        alive = False

    if alive:
        edges.iloc[i, 6] = '0_#107a06'
    elif dependent and not alive:
        edges.iloc[i, 6] = '1_#631da3'
    if not dependent and not alive:
        edges.iloc[i, 6] = '2_#730a1c'

nodes = edges[['victim', 'color']]
nodes.sort_values(by=['victim', 'color'], inplace=True)
victims = nodes['victim'].values
killers = edges['killer'].values

for killer in killers:
    if not killer in victims:
        data = {'victim': [killer], 'color': ['0_green']}
        df = pd.DataFrame(data)
        nodes = pd.concat([nodes, df])

nodes.sort_values(by=['victim', 'color'], inplace=True)
nodes.drop_duplicates('victim', keep='last', inplace=True)
nodes['color'] = nodes['color'].str.split('_').str[1]

descriptions = deaths.groupby(by=['universe', 'character']).aggregate({'description':'\n'.join})
descriptions.reset_index(inplace=True)
descriptions['character'] += " [" + descriptions['universe'] + "]"
descriptions.drop(columns=['universe'], inplace=True)
descriptions.rename(columns={"character":"victim"}, inplace=True)

nodes = pd.merge(nodes, descriptions, left_on=['victim'], right_on=['victim'], how='outer')

In [ ]:
profiles = pd.DataFrame(columns=['universe', 'character', 'profile'])

In [ ]:
def make_network(universe):
    net = Network(height="100vh", width="100%", bgcolor="#111111", directed=True, notebook=False)
    net.force_atlas_2based(overlap=1, damping=0.5)

    current_edges = edges[edges['universe'] == universe]
    current_nodes = nodes[nodes['victim'].str.endswith('[' + universe + ']')]

    sources, targets, direction = current_edges['killer'], current_edges['victim'], current_edges['direction']
    description = current_edges['description']
    edge_data = zip(sources, targets, direction, description)

    for i in range(len(current_nodes)):
        victim = current_nodes.iloc[i]['victim']
        color = current_nodes.iloc[i]['color']
        label = victim.split(" [")[0]
        description = current_nodes.iloc[i]['description']
        title = label + "\n\nCharacter Profile :" + "\n\n\n"

        if description == description:
            value = description.count('\n') + 1
            if value != 1:
                plural = 's'
            else:
                plural = ''

            title += str(value) + " Death" + plural + " :\n\n" + description + "\n\n"
        else:
            title += ''

        if victim != '':
            font = f"15px baskerville {color} sans-serif"
            net.add_node(victim, label=label, title=title, color=color, font=font)

    for src, dst, dir, des in edge_data:
        if src != '':
            if dir == 'direct':
                net.add_edge(src, dst, title=des, color='#e8ac87', width=1, dashes=False)
            elif dir == 'indirect':
                net.add_edge(src, dst, title=des, color='#cccccc', width=1, dashes=True)

    neighbor_map = net.get_adj_list()

    for node in net.nodes:
        value = len(neighbor_map[node['id']])
        if value != 0:
            if value != 1:
                plural = 's'
            else:
                plural = ''
            node['title'] += str(value) + " Victim" + plural + " :\n\n"
            node['title'] += "\n".join(sorted(neighbor_map[node['id']]))
            node['title'] = re.sub(' \[.*?\]', '', node['title'])
        node['value'] = value
    
    net.inherit_edge_colors(False)
    net.set_edge_smooth('dynamic')

    net.save_graph('Networks/' + universe + '.html')

    for node in net.nodes:
        universe = node['id'].split(' [')[1].split(']')[0]
        character = node['id'].split(' [')[0]
        profile = node['title']
        row = [universe, character, profile]
        profiles.loc[len(profiles)] = row

In [ ]:
for universe in kills['universe'].unique():
    make_network(universe)